In [1]:
# imports
import os
import glob
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from tqdm import tqdm
import os
import glob
import numpy as np
import pandas as pd
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from statsmodels.tsa.stattools import adfuller

In [2]:
# globals
DATA_DIR = ""   # <- Directory where parquet files are stored
WINDOW = 4320 # rolling window length (4320 = 3 days of 1-min bars set by default)
PVAL_THRESHOLD = 0.05 # cointegration significance level

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

def load_and_filter(folder, start="2024-05-01 00:00:00", end="2025-05-01 00:00:00"):
    """
    Load 1-minute crypto parquet files with full data coverage,
    filter by datetime range, and compute log prices & returns.
    """
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)
    data = {}

    for f in glob.glob(os.path.join(folder, "*_1m_bin_futures.parquet")):
        sym = os.path.basename(f).replace("_1m_bin_futures.parquet", "").replace("USDT", "")
        df = pd.read_parquet(f)

        # Convert to datetime if needed
        if "datetime" in df.columns:
            df["datetime"] = pd.to_datetime(df["datetime"])
        else:
            df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")

        # Filter by the specified time window
        df = df[(df["datetime"] >= start) & (df["datetime"] <= end)]
        df = df.set_index("datetime").sort_index()

        # Log prices and returns
        df["close"] = np.log(df["close"])
        df["open"] = np.log(df["open"])
        df["high"] = np.log(df["high"])
        df["low"] = np.log(df["low"])
        df["log_return"] = df["close"].diff()

        data[sym] = df

        print(f"Loaded {sym}, {len(df)} rows")

    return data


# Example usage
crypto_data = load_and_filter("", start="2024-05-01 00:00:00", end="2025-05-01 00:00:00")
print(f"✅ Loaded {len(crypto_data)} full symbols")

Loaded AAVE, 525601 rows
Loaded ADA, 525601 rows
Loaded APT, 525601 rows
Loaded ARB, 525601 rows
Loaded ATOM, 525601 rows
Loaded AVAX, 525601 rows
Loaded BCH, 525601 rows
Loaded BNB, 525601 rows
Loaded BTC, 525601 rows
Loaded DOGE, 525601 rows
Loaded DOT, 525601 rows
Loaded ENA, 525601 rows
Loaded ETC, 525601 rows
Loaded ETH, 525601 rows
Loaded HBAR, 525601 rows
Loaded LINK, 525601 rows
Loaded LTC, 525601 rows
Loaded NEAR, 525601 rows
Loaded SOL, 525601 rows
Loaded SUI, 525601 rows
Loaded TON, 525601 rows
Loaded TRX, 525601 rows
Loaded UNI, 525601 rows
Loaded WLD, 525601 rows
Loaded XLM, 525601 rows
Loaded XRP, 525601 rows
✅ Loaded 26 full symbols


In [7]:
# first round selection based on correlation (whole period)
# correlation matrix
def compute_correlation_matrix(crypto_data):
    """
    Compute correlation matrix of closing prices for all cryptos.
    Returns a DataFrame with symbols as both index and columns.
    """
    symbols = list(crypto_data.keys())
    close_prices = pd.DataFrame({sym: crypto_data[sym]["close"] for sym in symbols})
    corr_matrix = close_prices.corr()
    return corr_matrix
correlation_matrix = compute_correlation_matrix(crypto_data)
print(correlation_matrix)

          AAVE       ADA       APT       ARB      ATOM      AVAX       BCH  \
AAVE  1.000000  0.795344  0.287267 -0.116136 -0.048480  0.322662  0.205828   
ADA   0.795344  1.000000  0.265865  0.031820  0.297912  0.415695  0.451184   
APT   0.287267  0.265865  1.000000  0.739254  0.641260  0.884179  0.798965   
ARB  -0.116136  0.031820  0.739254  1.000000  0.899804  0.864401  0.824643   
ATOM -0.048480  0.297912  0.641260  0.899804  1.000000  0.837609  0.880414   
AVAX  0.322662  0.415695  0.884179  0.864401  0.837609  1.000000  0.907443   
BCH   0.205828  0.451184  0.798965  0.824643  0.880414  0.907443  1.000000   
BNB   0.723406  0.799381  0.544490  0.326498  0.476428  0.649551  0.637548   
BTC   0.843573  0.946783  0.240247 -0.096592  0.138178  0.330407  0.348356   
DOGE  0.787217  0.915396  0.539192  0.231368  0.401913  0.615882  0.603221   
DOT   0.176746  0.455486  0.687081  0.875746  0.949446  0.899393  0.896548   
ENA   0.335875  0.571381  0.660192  0.747169  0.850147  0.843024

In [8]:
# select those with correlation > 0.85
high_corr_pairs = []
threshold = 0.85
symbols = list(crypto_data.keys())
for i in range(len(symbols)):
    for j in range(i+1, len(symbols)):
        sym1, sym2 = symbols[i], symbols[j]
        corr = correlation_matrix.loc[sym1, sym2]
        if abs(corr) > threshold:
            high_corr_pairs.append((sym1, sym2))
            print(f"High correlation: {sym1} & {sym2} = {corr:.2f}")

High correlation: AAVE & SUI = 0.91
High correlation: AAVE & TRX = 0.87
High correlation: ADA & BTC = 0.95
High correlation: ADA & DOGE = 0.92
High correlation: ADA & HBAR = 0.94
High correlation: ADA & LTC = 0.90
High correlation: ADA & SUI = 0.85
High correlation: ADA & XLM = 0.97
High correlation: ADA & XRP = 0.91
High correlation: APT & AVAX = 0.88
High correlation: ARB & ATOM = 0.90
High correlation: ARB & AVAX = 0.86
High correlation: ARB & DOT = 0.88
High correlation: ARB & ETC = 0.91
High correlation: ARB & ETH = 0.93
High correlation: ARB & NEAR = 0.97
High correlation: ARB & WLD = 0.96
High correlation: ATOM & BCH = 0.88
High correlation: ATOM & DOT = 0.95
High correlation: ATOM & ENA = 0.85
High correlation: ATOM & ETC = 0.93
High correlation: AVAX & BCH = 0.91
High correlation: AVAX & DOT = 0.90
High correlation: AVAX & ETC = 0.94
High correlation: AVAX & UNI = 0.86
High correlation: BCH & DOT = 0.90
High correlation: BCH & ENA = 0.86
High correlation: BCH & ETC = 0.93
High

In [9]:
len(high_corr_pairs)  # number of high correlation pairs found

61

In [10]:

def rolling_cointegration(y, x, window=4320, adf_pval=0.05):
    """
    Rolling Engle–Granger cointegration test with beta estimation.
    Returns DataFrame with beta, ADF p-value, and spread.
    """
    y, x = y.align(x, join="inner")
    y, x = y.sort_index(), x.sort_index()

    results = []
    step = window // 3  # overlap control
    timestamps = y.index

    for i in range(0, len(timestamps) - window, step):
        start_time = timestamps[i]
        end_time = timestamps[i + window - 1]

        y_win = y.loc[start_time:end_time]
        x_win = x.loc[start_time:end_time]

        if len(y_win) < window or y_win.isna().any() or x_win.isna().any():
            continue

        model = OLS(y_win, add_constant(x_win)).fit()
        alpha, beta = model.params

        residuals = y_win - model.predict(add_constant(x_win))
        adf_p = adfuller(residuals)[1]
        corr = y_win.corr(x_win)

        results.append({
            "start": start_time,
            "end": end_time,
            "alpha": alpha,
            "beta": beta,
            "adf_p": adf_p,
            "cointegrated": adf_p <= adf_pval,
            "correlation": corr
        })

    return pd.DataFrame(results)

In [11]:
def prepare_all_pairs(crypto_data, list_of_pairs, window=4320, adf_pval=0.05):
    """
    Iterate through crypto pairs, compute rolling cointegration + correlation,
    and generate RL features.
    Returns: dict { (sym1, sym2): features_df }
    """
    pair_df = {}
    i = 1
    for sym1, sym2 in list_of_pairs:
        print(f"Processing pair: {sym1}, {sym2}. {i} of {len(list_of_pairs)}")
        i += 1
        y_ohlc = crypto_data[sym1]
        x_ohlc = crypto_data[sym2]

        # align close prices for cointegration
        df_close = pd.concat([y_ohlc["close"], x_ohlc["close"]], axis=1, join="inner").dropna()
        y_aligned, x_aligned = df_close.iloc[:, 0], df_close.iloc[:, 1]

        # Rolling-window Engle–Granger test
        coint_df = rolling_cointegration(y_aligned, x_aligned, window=window,
                                        adf_pval=adf_pval)
        pair_df[(sym1, sym2)] = coint_df

    return pair_df

In [12]:
coint_df = prepare_all_pairs(crypto_data, high_corr_pairs, window=WINDOW,
                             adf_pval=PVAL_THRESHOLD)

Processing pair: AAVE, SUI. 1 of 61
Processing pair: AAVE, TRX. 2 of 61
Processing pair: ADA, BTC. 3 of 61
Processing pair: ADA, DOGE. 4 of 61
Processing pair: ADA, HBAR. 5 of 61
Processing pair: ADA, LTC. 6 of 61
Processing pair: ADA, SUI. 7 of 61
Processing pair: ADA, XLM. 8 of 61
Processing pair: ADA, XRP. 9 of 61
Processing pair: APT, AVAX. 10 of 61
Processing pair: ARB, ATOM. 11 of 61
Processing pair: ARB, AVAX. 12 of 61
Processing pair: ARB, DOT. 13 of 61
Processing pair: ARB, ETC. 14 of 61
Processing pair: ARB, ETH. 15 of 61
Processing pair: ARB, NEAR. 16 of 61
Processing pair: ARB, WLD. 17 of 61
Processing pair: ATOM, BCH. 18 of 61
Processing pair: ATOM, DOT. 19 of 61
Processing pair: ATOM, ENA. 20 of 61
Processing pair: ATOM, ETC. 21 of 61
Processing pair: AVAX, BCH. 22 of 61
Processing pair: AVAX, DOT. 23 of 61
Processing pair: AVAX, ETC. 24 of 61
Processing pair: AVAX, UNI. 25 of 61
Processing pair: BCH, DOT. 26 of 61
Processing pair: BCH, ENA. 27 of 61
Processing pair: BCH,

In [14]:
#save dictionary into csv
for (sym1, sym2), df in coint_df.items():
    filename = f"{sym1}_{sym2}_bin_futures_window_cointegration.csv"
    df.to_csv(filename, index=False)
    print(f"Saved features for pair {sym1}-{sym2} to {filename}")

Saved features for pair AAVE-SUI to AAVE_SUI_bin_futures_window_cointegration.csv
Saved features for pair AAVE-TRX to AAVE_TRX_bin_futures_window_cointegration.csv
Saved features for pair ADA-BTC to ADA_BTC_bin_futures_window_cointegration.csv
Saved features for pair ADA-DOGE to ADA_DOGE_bin_futures_window_cointegration.csv
Saved features for pair ADA-HBAR to ADA_HBAR_bin_futures_window_cointegration.csv
Saved features for pair ADA-LTC to ADA_LTC_bin_futures_window_cointegration.csv
Saved features for pair ADA-SUI to ADA_SUI_bin_futures_window_cointegration.csv
Saved features for pair ADA-XLM to ADA_XLM_bin_futures_window_cointegration.csv
Saved features for pair ADA-XRP to ADA_XRP_bin_futures_window_cointegration.csv
Saved features for pair APT-AVAX to APT_AVAX_bin_futures_window_cointegration.csv
Saved features for pair ARB-ATOM to ARB_ATOM_bin_futures_window_cointegration.csv
Saved features for pair ARB-AVAX to ARB_AVAX_bin_futures_window_cointegration.csv
Saved features for pair AR

In [15]:
# selection of top-5 pairs per window based on correlation
top_pairs_per_window = {}
for pair, df in coint_df.items():
    for _, row in df.iterrows():
        window_key = (row["start"], row["end"])
        if window_key not in top_pairs_per_window:
            top_pairs_per_window[window_key] = []
        if row["cointegrated"]:
            top_pairs_per_window[window_key].append((pair, row["correlation"], row["beta"], row["alpha"], row["adf_p"]))
# keep top 5 pairs by correlation for each window
for window_key, pairs in top_pairs_per_window.items():
    # sort by absolute correlation
    pairs.sort(key=lambda x: abs(x[1]), reverse=True)  
    top_pairs_per_window[window_key] = pairs[:5]


In [ ]:
# make sure that there are 5 pairs for each window
for window_key, pairs in top_pairs_per_window.items():
    if len(pairs) < 5:
        print(f"Warning: Only {len(pairs)} pairs found for window {window_key}")
# not always 5 pairs

In [16]:
# calculate how many times each pair appears in top-5 across all windows
from collections import Counter
pair_counter = Counter()
for pairs in top_pairs_per_window.values():
    for pair_info in pairs:
        pair = pair_info[0]
        pair_counter[pair] += 1


In [17]:
len(pair_counter)
# each pair occurs at least once in top-5 across all windows

61

In [4]:
336/48

7.0

In [1]:
data = [
    (('ARB', 'ETC'), 60),
    (('NEAR', 'WLD'), 56),
    (('DOT', 'ETC'), 55),
    (('AVAX', 'DOT'), 54),
    (('ADA', 'HBAR'), 54),
    (('BTC', 'DOGE'), 53),
    (('BCH', 'ETC'), 51),
    (('ARB', 'WLD'), 50),
    (('ETC', 'NEAR'), 46),
    (('HBAR', 'XLM'), 45),
    (('ETC', 'ETH'), 44),
    (('ARB', 'NEAR'), 43),
    (('ATOM', 'DOT'), 42),
    (('DOGE', 'LINK'), 41),
    (('ATOM', 'ETC'), 41),
    (('XLM', 'XRP'), 40),
    (('ARB', 'DOT'), 39),
    (('AVAX', 'ETC'), 38),
    (('ARB', 'ETH'), 37),
    (('ARB', 'AVAX'), 36),
    (('ADA', 'XRP'), 35),
    (('APT', 'AVAX'), 35),
    (('ADA', 'DOGE'), 34),
    (('ARB', 'ATOM'), 32),
    (('ADA', 'LTC'), 31),
    (('HBAR', 'XRP'), 31),
    (('ADA', 'XLM'), 30),
    (('AVAX', 'BCH'), 27),
    (('DOGE', 'LTC'), 27),
    (('ADA', 'BTC'), 27),
    (('BNB', 'LINK'), 26),
    (('BTC', 'XRP'), 26),
    (('DOGE', 'XLM'), 25),
    (('AVAX', 'UNI'), 25),
    (('LINK', 'UNI'), 24),
    (('ATOM', 'BCH'), 23),
    (('LINK', 'LTC'), 23),
    (('LTC', 'XLM'), 22),
    (('ADA', 'SUI'), 22),
    (('BCH', 'DOT'), 22),
    (('ENA', 'UNI'), 22),
    (('BCH', 'ENA'), 21),
    (('ENA', 'ETC'), 21),
    (('DOT', 'ETH'), 20),
    (('AAVE', 'SUI'), 19),
    (('DOT', 'ENA'), 19),
    (('BTC', 'HBAR'), 19),
    (('BTC', 'SUI'), 18),
    (('BTC', 'XLM'), 18),
    (('DOGE', 'SUI'), 17),
    (('ETH', 'NEAR'), 16),
    (('HBAR', 'LTC'), 16),
    (('SUI', 'XLM'), 15),
    (('ATOM', 'ENA'), 15),
    (('BTC', 'LTC'), 14),
    (('TRX', 'XRP'), 9),
    (('BTC', 'TRX'), 8),
    (('SUI', 'TRX'), 7),
    (('TRX', 'XLM'), 7),
    (('AAVE', 'TRX'), 6),
    (('NEAR', 'TON'), 5)
]

total_sum = sum(value for _, value in data)
print(total_sum)


1784


In [5]:
1700*1440

2448000

In [18]:
pair_counter.most_common() 

[(('ARB', 'ETC'), 60),
 (('NEAR', 'WLD'), 56),
 (('DOT', 'ETC'), 55),
 (('AVAX', 'DOT'), 54),
 (('ADA', 'HBAR'), 54),
 (('BTC', 'DOGE'), 53),
 (('BCH', 'ETC'), 51),
 (('ARB', 'WLD'), 50),
 (('ETC', 'NEAR'), 46),
 (('HBAR', 'XLM'), 45),
 (('ETC', 'ETH'), 44),
 (('ARB', 'NEAR'), 43),
 (('ATOM', 'DOT'), 42),
 (('DOGE', 'LINK'), 41),
 (('ATOM', 'ETC'), 41),
 (('XLM', 'XRP'), 40),
 (('ARB', 'DOT'), 39),
 (('AVAX', 'ETC'), 38),
 (('ARB', 'ETH'), 37),
 (('ARB', 'AVAX'), 36),
 (('ADA', 'XRP'), 35),
 (('APT', 'AVAX'), 35),
 (('ADA', 'DOGE'), 34),
 (('ARB', 'ATOM'), 32),
 (('ADA', 'LTC'), 31),
 (('HBAR', 'XRP'), 31),
 (('ADA', 'XLM'), 30),
 (('AVAX', 'BCH'), 27),
 (('DOGE', 'LTC'), 27),
 (('ADA', 'BTC'), 27),
 (('BNB', 'LINK'), 26),
 (('BTC', 'XRP'), 26),
 (('DOGE', 'XLM'), 25),
 (('AVAX', 'UNI'), 25),
 (('LINK', 'UNI'), 24),
 (('ATOM', 'BCH'), 23),
 (('LINK', 'LTC'), 23),
 (('LTC', 'XLM'), 22),
 (('ADA', 'SUI'), 22),
 (('BCH', 'DOT'), 22),
 (('ENA', 'UNI'), 22),
 (('BCH', 'ENA'), 21),
 (('ENA',

In [ ]:
# calulcate how many unique crypto assets are involved in top pairs
unique_assets = set()
for pair in pair_counter.keys():
    unique_assets.update(pair)
len(unique_assets)  

25

In [21]:
import pandas as pd
import numpy as np

def build_full_features(crypto_data, top_pairs_per_window):
    # 1️⃣ Collect all unique symbols
    all_symbols = sorted({
        sym
        for pairs in top_pairs_per_window.values()
        for pair_info in pairs
        for sym in pair_info[0]
    })

    # 2️⃣ Initialize base DataFrame (timestamp index)
    full_df = pd.DataFrame(index=crypto_data["BTC"].index)
    full_df.index.name = "timestamp"

    # 3️⃣ Add log-price columns for each symbol
    for sym in all_symbols:
        df = crypto_data[sym][["close"]].rename(columns={"close": f"{sym}_close"})
        full_df = full_df.join(df, how="left")

    # 4️⃣ For each window and cointegrated pair, compute spreads and related stats
    for (start, end), pairs in top_pairs_per_window.items():
        mask = (full_df.index >= start) & (full_df.index <= end)

        for pair_info in pairs:
            (sym1, sym2), corr, beta, alpha, adf_p = pair_info

            spread_col = f"{sym1}_{sym2}_spread"
            beta_col = f"{sym1}_{sym2}_beta"
            alpha_col = f"{sym1}_{sym2}_alpha"
            adf_col = f"{sym1}_{sym2}_adf_p"
            corr_col = f"{sym1}_{sym2}_corr"

            # Initialize columns if they don't exist
            for col in [spread_col, beta_col, alpha_col, adf_col, corr_col]:
                if col not in full_df.columns:
                    full_df[col] = np.nan

            y = full_df.loc[mask, f"{sym1}_close"]
            x = full_df.loc[mask, f"{sym2}_close"]

            full_df.loc[mask, spread_col] = y - (alpha + beta * x)
            full_df.loc[mask, beta_col] = beta
            full_df.loc[mask, alpha_col] = alpha
            full_df.loc[mask, adf_col] = adf_p
            full_df.loc[mask, corr_col] = corr

    return full_df


In [ ]:
full_df = build_full_features(crypto_data, top_pairs_per_window)
print(full_df.shape)
full_df.head()
# 25 assets + 61 (pairs) spread features*5 = 330 columns total

C:\Users\isaen\AppData\Local\Temp\ipykernel_16112\3320542252.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[col] = np.nan
C:\Users\isaen\AppData\Local\Temp\ipykernel_16112\3320542252.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[col] = np.nan
C:\Users\isaen\AppData\Local\Temp\ipykernel_16112\3320542252.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

(525601, 330)


,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spread,TRX_XRP_beta,TRX_XRP_alpha,TRX_XRP_adf_p,TRX_XRP_corr,BTC_TRX_spread,BTC_TRX_beta,BTC_TRX_alpha,BTC_TRX_adf_p,BTC_TRX_corr
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-01 00:00:00,4.423169,-0.818257,2.166078,0.021370,2.136176,3.486794,6.072814,6.358864,11.013869,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-01 00:01:00,4.422809,-0.819164,2.164012,0.020195,2.138418,3.485937,6.071938,6.357825,11.014249,-2.014703,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-01 00:02:00,4.421368,-0.819845,2.163553,0.018527,2.136294,3.485630,6.070992,6.357634,11.012786,-2.016504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-01 00:03:00,4.422929,-0.818937,2.167223,0.020097,2.136176,3.486304,6.072422,6.357409,11.013483,-2.015528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-01 00:04:00,4.422569,-0.818937,2.166651,0.019508,2.135349,3.485937,6.071476,6.357201,11.012911,-2.015303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # creation a DataFrame with all 1-min timestamps and prices for all symbols in cointegrated pairs, and window-specific spreads

# #Get all unique symbols from cointegrated pairs
# all_symbols = set()
# for pairs in top_pairs_per_window.values():
#     for pair_info in pairs:
#         sym1, sym2 = pair_info[0]
#         all_symbols.add(sym1)
#         all_symbols.add(sym2)
# all_symbols = sorted(list(all_symbols))

# full_df = pd.DataFrame({"timestamp": crypto_data["BTC"]["timestamp"]})  # index of any crypto is in fact timestamps - take any

# # price columns for each symbol
# for sym in all_symbols:
#     df = crypto_data[sym].copy()
#     full_df = full_df.merge(df[["timestamp", "close"]], on="timestamp", how="left")
#     full_df = full_df.rename(columns={"close": f"{sym}_close"})

# # for each window and pair, assign beta, alpha, and calculate spread for that window
# for (start, end), pairs in top_pairs_per_window.items():
#     mask = (full_df["timestamp"] >= start) & (full_df["timestamp"] <= end)
#     for pair_info in pairs:
#         sym1, sym2 = pair_info[0]
#         corr, beta, alpha, adf_p = pair_info[1:]
#         spread_col = f"{sym1}_{sym2}_spread"
#         if spread_col not in full_df.columns:
#             full_df[spread_col] = np.nan
#         y = full_df.loc[mask, f"{sym1}_close"]
#         x = full_df.loc[mask, f"{sym2}_close"]
#         full_df.loc[mask, spread_col] = y - (alpha + beta * x)
#         full_df.loc[mask, f"{sym1}_{sym2}_beta"] = beta
#         full_df.loc[mask, f"{sym1}_{sym2}_alpha"] = alpha
#         full_df.loc[mask, f"{sym1}_{sym2}_adf_p"] = adf_p
#         full_df.loc[mask, f"{sym1}_{sym2}_corr"] = corr

# full_df.head()
# # 55 (pairs) spread features*5 + 23 (assets) price features + 1 timestamp = 299 columns total

TypeError: '>=' not supported between instances of 'numpy.ndarray' and 'Timestamp'

AAVE_close        4.423169
ADA_close        -0.818257
APT_close         2.166078
ARB_close         0.021370
ATOM_close        2.136176
                    ...   
BTC_TRX_spread         NaN
BTC_TRX_beta           NaN
BTC_TRX_alpha          NaN
BTC_TRX_adf_p          NaN
BTC_TRX_corr           NaN
Name: 2024-05-01 00:00:00, Length: 330, dtype: float64

In [45]:
na_counts = full_df.notna().sum(axis=1)

# Filter rows where n_NaN is NOT in the allowed set
allowed = [25, 30, 35, 40, 45, 50]
problematic_rows = full_df[~na_counts.isin(allowed)]

problematic_rows.head()


,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spread,TRX_XRP_beta,TRX_XRP_alpha,TRX_XRP_adf_p,TRX_XRP_corr,BTC_TRX_spread,BTC_TRX_beta,BTC_TRX_alpha,BTC_TRX_adf_p,BTC_TRX_corr
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-02 00:00:00,4.426044,-0.800510,2.187062,0.027907,2.166651,3.512053,6.049828,6.329382,10.973357,-2.040221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:01:00,4.425326,-0.800732,2.187174,0.027031,2.164816,3.511963,6.049969,6.328740,10.974096,-2.042146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:02:00,4.425565,-0.799175,2.187174,0.028587,2.165390,3.513007,6.051783,6.329436,10.975292,-2.039836,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:03:00,4.426880,-0.798063,2.187286,0.030820,2.166994,3.512291,6.051972,6.330505,10.975792,-2.038069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:04:00,4.430221,-0.796510,2.189976,0.033822,2.168711,3.514169,6.056362,6.332107,10.977635,-2.035846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
problematic_rows

,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spread,TRX_XRP_beta,TRX_XRP_alpha,TRX_XRP_adf_p,TRX_XRP_corr,BTC_TRX_spread,BTC_TRX_beta,BTC_TRX_alpha,BTC_TRX_adf_p,BTC_TRX_corr
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-02 00:00:00,4.426044,-0.800510,2.187062,0.027907,2.166651,3.512053,6.049828,6.329382,10.973357,-2.040221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:01:00,4.425326,-0.800732,2.187174,0.027031,2.164816,3.511963,6.049969,6.328740,10.974096,-2.042146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:02:00,4.425565,-0.799175,2.187174,0.028587,2.165390,3.513007,6.051783,6.329436,10.975292,-2.039836,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:03:00,4.426880,-0.798063,2.187286,0.030820,2.166994,3.512291,6.051972,6.330505,10.975792,-2.038069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:04:00,4.430221,-0.796510,2.189976,0.033822,2.168711,3.514169,6.056362,6.332107,10.977635,-2.035846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-29 23:55:00,5.109998,-0.362549,1.692179,-1.115657,1.475678,3.069169,5.894816,6.398179,11.453195,-1.745716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-29 23:56:00,5.109636,-0.363268,1.692234,-1.115962,1.475907,3.069308,5.894981,6.397813,11.452964,-1.746174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-29 23:57:00,5.110179,-0.363412,1.692565,-1.115352,1.476592,3.070376,5.895944,6.397829,11.453304,-1.745945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
full_df[~(full_df.isna().sum(axis=1) == (330-25-5*5))]

,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spread,TRX_XRP_beta,TRX_XRP_alpha,TRX_XRP_adf_p,TRX_XRP_corr,BTC_TRX_spread,BTC_TRX_beta,BTC_TRX_alpha,BTC_TRX_adf_p,BTC_TRX_corr
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-02 00:00:00,4.426044,-0.800510,2.187062,0.027907,2.166651,3.512053,6.049828,6.329382,10.973357,-2.040221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:01:00,4.425326,-0.800732,2.187174,0.027031,2.164816,3.511963,6.049969,6.328740,10.974096,-2.042146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:02:00,4.425565,-0.799175,2.187174,0.028587,2.165390,3.513007,6.051783,6.329436,10.975292,-2.039836,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:03:00,4.426880,-0.798063,2.187286,0.030820,2.166994,3.512291,6.051972,6.330505,10.975792,-2.038069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02 00:04:00,4.430221,-0.796510,2.189976,0.033822,2.168711,3.514169,6.056362,6.332107,10.977635,-2.035846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-29 23:56:00,5.109636,-0.363268,1.692234,-1.115962,1.475907,3.069308,5.894981,6.397813,11.452964,-1.746174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-29 23:57:00,5.110179,-0.363412,1.692565,-1.115352,1.476592,3.070376,5.895944,6.397829,11.453304,-1.745945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-29 23:58:00,5.109817,-0.362980,1.691331,-1.115352,1.476135,3.070469,5.895834,6.397763,11.453290,-1.745601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
full_df.to_csv("bin_futures_historical_pairs_with_spreads.csv", index=False)

In [51]:
l = [full_df.iloc[i].isna().sum() for i in range(len(full_df))]

In [ ]:
# unique values in l
set(l)
# 230 = 330 - 50*2 (overlap) 

{230, 235, 240, 245, 250, 255, 260, 265, 270, 280, 305}

Index(['AAVE_close', 'ADA_close', 'APT_close', 'ARB_close', 'ATOM_close',
       'AVAX_close', 'BCH_close', 'BNB_close', 'BTC_close', 'DOGE_close',
       ...
       'TRX_XRP_spread', 'TRX_XRP_beta', 'TRX_XRP_alpha', 'TRX_XRP_adf_p',
       'TRX_XRP_corr', 'BTC_TRX_spread', 'BTC_TRX_beta', 'BTC_TRX_alpha',
       'BTC_TRX_adf_p', 'BTC_TRX_corr'],
      dtype='object', length=330)

In [54]:
full_df[full_df.isna().sum(axis=1) == 230]

,AAVE_close,ADA_close,APT_close,ARB_close,ATOM_close,AVAX_close,BCH_close,BNB_close,BTC_close,DOGE_close,...,TRX_XRP_spread,TRX_XRP_beta,TRX_XRP_alpha,TRX_XRP_adf_p,TRX_XRP_corr,BTC_TRX_spread,BTC_TRX_beta,BTC_TRX_alpha,BTC_TRX_adf_p,BTC_TRX_corr
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-07-06 00:00:00,4.378520,-1.052970,1.710730,-0.475137,1.739062,3.221113,5.783025,6.209334,10.943623,-2.253985,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-06 00:01:00,4.375631,-1.054117,1.709464,-0.476102,1.737303,3.220634,5.781638,6.208691,10.943123,-2.253509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-06 00:02:00,4.377516,-1.053257,1.710730,-0.474172,1.738534,3.221831,5.783056,6.209495,10.943651,-2.250942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-06 00:03:00,4.375002,-1.054978,1.708015,-0.477713,1.736071,3.219076,5.780280,6.207442,10.942666,-2.255225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-06 00:04:00,4.375757,-1.054404,1.709102,-0.476907,1.738007,3.219755,5.781052,6.207060,10.941891,-2.255415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-13 23:55:00,5.091109,-0.354249,1.637753,-1.069442,1.362002,2.917554,5.786621,6.361579,11.303464,-1.801810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-13 23:56:00,5.092584,-0.353537,1.638647,-1.068568,1.362770,2.917717,5.786683,6.361769,11.303316,-1.801567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-13 23:57:00,5.091662,-0.354249,1.637870,-1.069151,1.361745,2.916635,5.786744,6.361855,11.303028,-1.801689,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# make sure that there are 5 pairs for each window
for window_key, pairs in top_pairs_per_window.items():
    if len(pairs) < 5:
        print(f"Warning: Only {len(pairs)} pairs found for window {window_key}")
# not always 5 pairs

In [67]:
top_pairs_per_window[(pd.Timestamp('2024-07-06 00:00:00'), pd.Timestamp('2024-07-08 23:59:00'))]

[(('ADA', 'LTC'),
  0.9732799084897753,
  1.2396422632353334,
  -6.170703490664128,
  0.001161524286787201),
 (('LINK', 'LTC'),
  0.9518375658442225,
  1.196781338714112,
  -2.4268112380017133,
  0.017729384664906894),
 (('HBAR', 'XRP'),
  0.9485829619792489,
  1.151403309772805,
  -1.7357516556295898,
  0.018772204658598843),
 (('BTC', 'DOGE'),
  0.9475848029186897,
  0.5082024015560704,
  12.078883578173878,
  0.007458213238889828),
 (('BTC', 'XLM'),
  0.935083325580816,
  0.5203410772399053,
  12.219190381233517,
  0.006323753174875367)]

In [62]:
top_pairs_per_window.keys()

dict_keys([(Timestamp('2024-05-01 00:00:00'), Timestamp('2024-05-03 23:59:00')), (Timestamp('2024-05-02 00:00:00'), Timestamp('2024-05-04 23:59:00')), (Timestamp('2024-05-03 00:00:00'), Timestamp('2024-05-05 23:59:00')), (Timestamp('2024-05-04 00:00:00'), Timestamp('2024-05-06 23:59:00')), (Timestamp('2024-05-05 00:00:00'), Timestamp('2024-05-07 23:59:00')), (Timestamp('2024-05-06 00:00:00'), Timestamp('2024-05-08 23:59:00')), (Timestamp('2024-05-07 00:00:00'), Timestamp('2024-05-09 23:59:00')), (Timestamp('2024-05-08 00:00:00'), Timestamp('2024-05-10 23:59:00')), (Timestamp('2024-05-09 00:00:00'), Timestamp('2024-05-11 23:59:00')), (Timestamp('2024-05-10 00:00:00'), Timestamp('2024-05-12 23:59:00')), (Timestamp('2024-05-11 00:00:00'), Timestamp('2024-05-13 23:59:00')), (Timestamp('2024-05-12 00:00:00'), Timestamp('2024-05-14 23:59:00')), (Timestamp('2024-05-13 00:00:00'), Timestamp('2024-05-15 23:59:00')), (Timestamp('2024-05-14 00:00:00'), Timestamp('2024-05-16 23:59:00')), (Timestam

In [64]:
top_pairs_per_window[(pd.Timestamp('2024-05-01 00:00:00'), pd.Timestamp('2024-05-03 23:59:00'))]

[(('AVAX', 'ETC'),
  0.9741800335098781,
  1.2010378687797196,
  -0.38587014779241713,
  0.038911554622404136),
 (('XLM', 'XRP'),
  0.9594458707339523,
  0.7275602589976646,
  -1.7309388665873797,
  0.00394629244133999),
 (('ETC', 'ETH'),
  0.9444983486845285,
  1.2561219068345864,
  -6.806165235426221,
  0.046266802293490464),
 (('ADA', 'XRP'),
  0.9427801627025159,
  1.0219154351046322,
  -0.11805425039149098,
  0.0168253001015466),
 (('ADA', 'LTC'),
  0.9414120954592604,
  1.2107959157928638,
  -6.099804334369608,
  0.04899689205298267)]

In [75]:
# check if there are na in close prices for full_df
for col in full_df.columns:
    if col.endswith("_close"):
        if full_df[col].isna().any():
            print(f"Column {col} has NaN values")
        else:
            print(f"Column {col} is complete")


Column AAVE_close is complete
Column ADA_close is complete
Column APT_close is complete
Column ARB_close is complete
Column ATOM_close is complete
Column AVAX_close is complete
Column BCH_close is complete
Column BNB_close is complete
Column BTC_close is complete
Column DOGE_close is complete
Column DOT_close is complete
Column ENA_close is complete
Column ETC_close is complete
Column ETH_close is complete
Column HBAR_close is complete
Column LINK_close is complete
Column LTC_close is complete
Column NEAR_close is complete
Column SUI_close is complete
Column TON_close is complete
Column TRX_close is complete
Column UNI_close is complete
Column WLD_close is complete
Column XLM_close is complete
Column XRP_close is complete
